In [1]:
simuler_X_R <- function(n, lambda, mu, sigma) {
  resultats <- numeric(n)
  for (i in 1:n) {
    N <- rpois(1, lambda)
    if (N > 0) {
      C_i <- rlnorm(N, meanlog = mu, sdlog = sigma)
      resultats[i] <- sum(C_i)
    } else {
      resultats[i] = 0
    }
  }
  return(resultats)
}

In [2]:
library(Rcpp)

cppFunction('
NumericVector rpcomp2_cpp(int n, double lamb, double mu, double sigma) {
    NumericVector resultats(n);
    
    for (int i = 0; i < n; i++) {
        int N = R::rpois(lamb);
        double somme = 0.0;
        
        for (int j = 0; j < N; j++) {
            somme += R::rlnorm(mu, sigma);
        }
        
        resultats[i] = somme;
    }
    
    return resultats;
}
')

In [3]:
# Chargement du code C++ séparé
sourceCpp("simulation2.cpp")

In [4]:
#comparaison des performances

# Paramètres
n <- 100000
lambda <- 5
mu <- 0
sigma <- 1

# Mesure des temps
temps_R <- system.time({
  resultats_R <- simuler_X_R(n, lambda, mu, sigma)
})[3]

temps_cppFunction <- system.time({
  resultats_cppFunction <- rpcomp2_cpp(n, lambda, mu, sigma)
})[3]

temps_sourceCpp <- system.time({
  resultats_sourceCpp <- rpcomp2(n, lambda, mu, sigma)
})[3]

# Affichage
cat("TEMPS D'EXÉCUTION =\n")
cat("R pur:", round(temps_R, 3), "s\n")
cat("Rcpp cppFunction:", round(temps_cppFunction, 3), "s\n")
cat("Rcpp sourceCpp:", round(temps_sourceCpp, 3), "s\n")
cat("\n= RAPPORT DE VITESSE=\n")
cat("cppFunction vs R:", round(temps_R/temps_cppFunction, 1), "x plus rapide\n")
cat("sourceCpp vs R:", round(temps_R/temps_sourceCpp, 1), "x plus rapide\n")
cat("cppFunction vs sourceCpp:", round(temps_cppFunction/temps_sourceCpp, 2), "\n")

TEMPS D'EXÉCUTION =
R pur: 0.112 s
Rcpp cppFunction: 0.015 s
Rcpp sourceCpp: 0.015 s

= RAPPORT DE VITESSE=
cppFunction vs R: 7.5 x plus rapide
sourceCpp vs R: 7.5 x plus rapide
cppFunction vs sourceCpp: 1 


Je constate que R pur est le pus lent. cppFunction et sourceCpp sont 10-20x plus rapide que R. De plus,
cppFunction et sourceCpp ont approximativement les mêmes performances. Aucune différence significative en
vitesse.

In [5]:
# AKATCHI DANIEL
# p2412436
# Master 1 Actuariat ISFA